In [1]:
import pandas as pd

# Load the datasets
applications_df = pd.read_csv('cleaned_applications.csv')
branches_df = pd.read_csv('cleaned_branches.csv')
customers_df = pd.read_csv('cleaned_customers.csv')
defaults_df = pd.read_csv('cleaned_defaults.csv')
loans_df = pd.read_csv('cleaned_loans.csv')
transactions_df = pd.read_csv('cleaned_transactions.csv')

print("Datasets loaded successfully.")

Datasets loaded successfully.


In [2]:
# 1. Group by Customer_ID and calculate transaction count and mean amount
customer_transaction_summary = transactions_df.groupby('Customer_ID').agg(
    transaction_count=('Transaction_ID', 'count'),
    mean_transaction_amount=('Amount', 'mean')
).reset_index()

# 2. Calculate median and IQR for transaction count and mean transaction amount
median_count = customer_transaction_summary['transaction_count'].median()
q1_count = customer_transaction_summary['transaction_count'].quantile(0.25)
q3_count = customer_transaction_summary['transaction_count'].quantile(0.75)
iqr_count = q3_count - q1_count

median_amount = customer_transaction_summary['mean_transaction_amount'].median()
q1_amount = customer_transaction_summary['mean_transaction_amount'].quantile(0.25)
q3_amount = customer_transaction_summary['mean_transaction_amount'].quantile(0.75)
iqr_amount = q3_amount - q1_amount

# Define bounds for identifying irregular patterns using 1.5 * IQR
lower_bound_count = q1_count - 1.5 * iqr_count
upper_bound_count = q3_count + 1.5 * iqr_count

lower_bound_amount = q1_amount - 1.5 * iqr_amount
upper_bound_amount = q3_amount + 1.5 * iqr_amount

# 3. Identify customers with irregular repayment patterns
irregular_customers_df = customer_transaction_summary[
    (customer_transaction_summary['transaction_count'] < lower_bound_count) |
    (customer_transaction_summary['transaction_count'] > upper_bound_count) |
    (customer_transaction_summary['mean_transaction_amount'] < lower_bound_amount) |
    (customer_transaction_summary['mean_transaction_amount'] > upper_bound_amount)
]

# 4. Store the customer IDs of those identified with irregular patterns
irregular_customer_ids = irregular_customers_df['Customer_ID'].tolist()

print("Identified customers with irregular repayment patterns.")

Identified customers with irregular repayment patterns.


In [3]:
# 1. Filter for penalty payments
penalty_transactions_df = transactions_df[transactions_df['Payment_Type'] == 'Penalty']

# 2. Calculate the sum of 'Amount' for penalty transactions
total_penalty_amount = penalty_transactions_df['Amount'].sum()

# 3. Calculate the sum of 'Amount' for all transactions
total_transaction_amount = transactions_df['Amount'].sum()

# 4. Calculate the proportion of penalty payments
penalty_proportion = (total_penalty_amount / total_transaction_amount) * 100

# 5. Print the calculated penalty proportion
print(f"Proportion of penalty payments: {penalty_proportion:.2f}%")

Proportion of penalty payments: 50.06%


In [4]:
# 1. Merge the transactions_df DataFrame with the loans_df DataFrame on the 'Loan_ID' column
merged_df = pd.merge(transactions_df, loans_df, on='Loan_ID')

# 2. Group the merged DataFrame by 'Loan_Status' and calculate the mean of the 'Amount' column
average_transaction_amount_by_status = merged_df.groupby('Loan_Status')['Amount'].mean()

# 3. Print the calculated average transaction amount for each loan status
print("Average transaction amount by Loan Status:")
print(average_transaction_amount_by_status)

Average transaction amount by Loan Status:
Loan_Status
Active     25502.802129
Closed     25496.166516
Overdue    25490.255176
Name: Amount, dtype: float64


In [5]:
# 1. Summarize findings on irregular repayment patterns
print("Summary of Transaction Pattern Analysis:")
print("\n1. Irregular Repayment Patterns:")
print(f"- Customers with irregular repayment patterns were identified as outliers in transaction count and mean transaction amount.")
print(f"- Outliers were determined using the 1.5 * IQR rule based on the distribution of transaction counts and average transaction amounts across all customers.")
print(f"- A total of {len(irregular_customers_df)} customers were identified with irregular patterns.")
print(f"- The median transaction count was {median_count}, and the median mean transaction amount was {median_amount:.2f}.")


# 2. Summarize findings on penalty payments proportion
print("\n2. Proportion of Penalty Payments:")
print(f"- The proportion of penalty payments out of the total transaction amounts was calculated.")
print(f"- The calculated proportion is {penalty_proportion:.2f}%. This indicates that approximately half of the total transaction value comes from penalty payments.")

# 3. Summarize findings on transaction amounts for overdue vs. non-overdue loans
print("\n3. Transaction Amounts for Overdue vs. Non-Overdue Loans:")
print(f"- The average transaction amount was compared across different loan statuses (Active, Closed, Overdue).")
print(f"- The average transaction amounts are:")
print(f"  - Active Loans: {average_transaction_amount_by_status.get('Active', 0):.2f}")
print(f"  - Closed Loans: {average_transaction_amount_by_status.get('Closed', 0):.2f}")
print(f"  - Overdue Loans: {average_transaction_amount_by_status.get('Overdue', 0):.2f}")
print(f"- The average transaction amounts are relatively similar across all loan statuses, with Overdue loans showing a slightly lower average transaction amount.")

Summary of Transaction Pattern Analysis:

1. Irregular Repayment Patterns:
- Customers with irregular repayment patterns were identified as outliers in transaction count and mean transaction amount.
- Outliers were determined using the 1.5 * IQR rule based on the distribution of transaction counts and average transaction amounts across all customers.
- A total of 2549 customers were identified with irregular patterns.
- The median transaction count was 8.0, and the median mean transaction amount was 25492.08.

2. Proportion of Penalty Payments:
- The proportion of penalty payments out of the total transaction amounts was calculated.
- The calculated proportion is 50.06%. This indicates that approximately half of the total transaction value comes from penalty payments.

3. Transaction Amounts for Overdue vs. Non-Overdue Loans:
- The average transaction amount was compared across different loan statuses (Active, Closed, Overdue).
- The average transaction amounts are:
  - Active Loans: 2